# Used cars price prediction

## Research objective

Многие знают про маркетплейсы где продаются б/у вещи, на которых есть возможность недорого купить качественную и полезную вещь. Но всегда волнует вопрос - кто и как устанавливает цену, и какие его характеристики больше всего влияют на итоговую стоимость продажи?! Вопрос становиться особо актуальным, если речь идет про дорогие товары, например про автомобили!
Предлагаем вам принять участие в Мастерской, в рамках которой вы сможете поработать с данными о продажах автомобилей на вторичном рынке. Целью вашего проекта будет разработанная модель предсказания стоимости автомобиля на вторичном рынке.

## Data pre-processing

### Load libraries and files

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import sklearn

import optuna
from optuna.samplers import TPESampler

from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from scipy import stats as st

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data_train = pd.read_csv(r'C:\Users\90sds\Desktop\Python\Yandex\Masterskaya\data\train.csv')

In [3]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          432193 non-null  object 
 2   model         432113 non-null  object 
 3   trim          431899 non-null  object 
 4   body          429843 non-null  object 
 5   transmission  388775 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     430831 non-null  float64
 9   odometer      440167 non-null  float64
 10  color         439650 non-null  object 
 11  interior      439650 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB


In [4]:
display(data_train.head())

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,Ford,Edge,SEL,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,Tue Jun 02 2015 02:30:00 GMT-0700 (PDT)
1,2014,Ford,Fusion,SE,Sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,Wed Feb 25 2015 02:00:00 GMT-0800 (PST)
2,2012,Nissan,Sentra,2.0 SL,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,Wed Jun 10 2015 02:30:00 GMT-0700 (PDT)
3,2003,HUMMER,H2,Base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,Wed Jun 17 2015 03:00:00 GMT-0700 (PDT)
4,2007,Ford,Fusion,SEL,Sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,Tue Feb 03 2015 04:00:00 GMT-0800 (PST)


### Data cleaning

In [5]:
data_train['saledate'] = pd.to_datetime(data_train['saledate'], format='%a %b %d %Y %H:%M:%S %Z', exact=False)
data_train['sell_year'] = data_train['saledate'].dt.year
data_train['sell_month'] = data_train['saledate'].dt.month
data_train['sell_day'] = data_train['saledate'].dt.day
data_train['age'] = data_train['sell_year'] - data_train['year']

converting sales date to the date format

In [6]:
for x in data_train.select_dtypes(include="object"):
    data_train[x] = data_train[x].str.lower()
    
make_dupl = [' tk', ' truck', ' ', '-b', '-benz']
for y in make_dupl:
    data_train['make'] = data_train['make'].str.replace(y, '')

data_train['make'] = data_train['make'].str.replace('mercedesenz', 'mercedes')    
data_train['make'] = data_train['make'].str.replace('vw', 'volkswagen')
data_train['make'].fillna('other', inplace=True)

In [7]:
display(data_train['make'].sort_values().unique())
print()
display(data_train['make'].value_counts())

array(['acura', 'airstream', 'astonmartin', 'audi', 'bentley', 'bmw',
       'buick', 'cadillac', 'chevrolet', 'chrysler', 'daewoo', 'dodge',
       'dot', 'ferrari', 'fiat', 'fisker', 'ford', 'geo', 'gmc', 'honda',
       'hummer', 'hyundai', 'infiniti', 'isuzu', 'jaguar', 'jeep', 'kia',
       'lamborghini', 'landrover', 'lexus', 'lincoln', 'maserati',
       'mazda', 'mercedes', 'mercury', 'mini', 'mitsubishi', 'nissan',
       'oldsmobile', 'other', 'plymouth', 'pontiac', 'porsche', 'ram',
       'rolls-royce', 'saab', 'saturn', 'scion', 'smart', 'subaru',
       'suzuki', 'tesla', 'toyota', 'volkswagen', 'volvo'], dtype=object)

ford           74070
chevrolet      47818
nissan         42866
toyota         31711
dodge          24295
honda          21600
hyundai        17187
bmw            16254
kia            14325
chrysler       13796
mercedes       13408
infiniti       12123
jeep           12107
volkswagen      9901
lexus           9363
gmc             8368
other           8043
mazda           6704
cadillac        5958
acura           4679
audi            4635
lincoln         4541
buick           4023
subaru          4019
pontiac         3596
ram             3592
mitsubishi      3413
volvo           2958
mini            2473
saturn          2247
mercury         1561
landrover       1476
scion           1338
jaguar          1100
porsche         1094
suzuki           846
fiat             672
hummer           597
saab             384
smart            311
oldsmobile       294
isuzu            163
maserati         112
bentley           91
astonmartin       23
plymouth          23
tesla             17
ferrari      

Cleaning "make column"

In [8]:
model_dupl = [' sports wagon', 'v8 ', ' quattro', ' gtc', ' gt', ' speed', ' supersports', ' series', ' gran turismo',
              ' gran coupe', 'activehybrid ', ' coupe', ' hybrid', ' esv', ' ext', ' wagon', ' limited', '3500hd', ' sport', 
             ' classic', ' 2500hd', ' 1500hd', 'black diamond ', ' 1500', ' 2500', ' stingray', 's-10 ', ' maxx', '/z71',
             ' cargo', ' 3500', ' ev', ' cruiser', 'pickup', ' van', '-150', '-250', '-350', '-series', ' victoria', 'energi',
             ' heritage', ' super duty', ' st', ' hundred', ' connect', ' svt cobra', ' svt lightning', '-450', ' el', 'trac',
             ' xl', ' xuv', ' denali', ' crosstour', ' del sol', ' touring', ' sut', ' fe', ' convertible', ' sedan', ' ii',
             ' roveroque', ' 250', ' 350', ' 330', ' 400', ' 430', ' 450h', ' 460', ' 470', ' 250h', ' 300', ' 200h', ' 300h',
             ' 600h l', ' 570', ' 450', ' viii', ' car', ' truck', ' miata', 'mazdaspeed ', ' mazdaspeed', '-class', ' marquis',
             'cooper ', ' spyder', ' passenger', ' select', ' note', ' crosstt', ' ciera', ' supreme', ' royale', ' am', ' prix',
             ' sv6', '/', ' tradesman', 'b9 ', ' wrx', ' crosstrek', ' vitara', ' solara', ' plug-in', ' 2', 'new ', ' gli']
for y in model_dupl:
    data_train['model'] = data_train['model'].str.replace(y, '')

data_train['model'] = data_train['model'].str.replace('rs 7', 's7')
data_train['model'] = data_train['model'].str.replace('r8', 's8')
data_train['model'] = data_train['model'].str.replace('rs 6', 's6')
data_train['model'] = data_train['model'].str.replace('cabriolet', 'tt')
data_train['model'] = data_train['model'].str.replace('tts', 'tt')
data_train['model'] = data_train['model'].str.replace('rs 4', 's4')
data_train['model'] = data_train['model'].str.replace('sq5', 'q5')
data_train['model'] = data_train['model'].str.replace('rs 5', 's5')
data_train['model'] = data_train['model'].str.replace('tt rs', 'tt')
data_train['model'] = data_train['model'].str.replace('continental flying spur', 'flying spur')
data_train['model'] = data_train['model'].str.replace('x6 m', 'x6')
data_train['model'] = data_train['model'].str.replace('750i', '7')
data_train['model'] = data_train['model'].str.replace('750li', '7')
data_train['model'] = data_train['model'].str.replace('320i', '3')
data_train['model'] = data_train['model'].str.replace('x5 m', 'x5')
data_train['model'] = data_train['model'].str.replace('750lxi', '7')
data_train['model'] = data_train['model'].str.replace('z4 m', 'z4')
data_train['model'] = data_train['model'].str.replace('328i', '3')
data_train['model'] = data_train['model'].str.replace('323i', '3')
data_train['model'] = data_train['model'].str.replace('x6 m', 'x6')
data_train['model'] = data_train['model'].str.replace('cts-v', 'cts')
data_train['model'] = data_train['model'].str.replace('sts-v', 'sts')
data_train['model'] = data_train['model'].str.replace('s-10', 'blazer')
data_train['model'] = data_train['model'].str.replace('s10', 'blazer')
data_train['model'] = data_train['model'].str.replace('c/k', 'silverado')
data_train['model'] = data_train['model'].str.replace('uplandr', 'uplander')
data_train.loc[(data_train['make'] == 'chevrolet') & (data_train['model'] == '2500'), 'model'] = 'silverado'
data_train.loc[(data_train['make'] == 'chevrolet') & (data_train['model'] == 'g2500'), 'model'] = 'silverado'
data_train.loc[(data_train['make'] == 'chevrolet') & (data_train['model'] == 'g3500'), 'model'] = 'silverado'
data_train.loc[(data_train['make'] == 'chevrolet') & (data_train['model'] == '3500'), 'model'] = 'silverado'
data_train.loc[(data_train['make'] == 'chevrolet') & (data_train['model'] == 'g1500'), 'model'] = 'silverado'
data_train.loc[(data_train['make'] == 'chevrolet') & (data_train['model'] == '1500'), 'model'] = 'silverado'
data_train.loc[(data_train['make'] == 'chevrolet') & (data_train['model'] == 'capt'), 'model'] = 'captiva'
data_train.loc[(data_train['make'] == 'chevrolet') & (data_train['model'] == 'ss'), 'model'] = 'ssr'
data_train.loc[(data_train['make'] == 'chrysler') & (data_train['model'] == '300m'), 'model'] = '300'
data_train.loc[(data_train['make'] == 'chrysler') & (data_train['model'] == 'twn&country'), 'model'] = 'twn/cntry'
data_train.loc[(data_train['make'] == 'chrysler') & (data_train['model'] == 'town'), 'model'] = 'twn/cntry'
data_train.loc[(data_train['make'] == 'chrysler') & (data_train['model'] == 'town and country'), 'model'] = 'twn/cntry'
data_train.loc[(data_train['make'] == 'dodge') & (data_train['model'] == 'grand'), 'model'] = 'gr'
data_train.loc[(data_train['make'] == 'dodge') & (data_train['model'] == 'grand caravan'), 'model'] = 'gr'
data_train.loc[(data_train['make'] == 'dodge') & (data_train['model'] == 'ram3500'), 'model'] = 'ram'
data_train.loc[data_train['make'] == 'ferrari', 'model'] = 'ferrari'
data_train.loc[data_train['make'] == 'fiat', 'model'] = '500'
data_train.loc[data_train['make'] == 'geo', 'model'] = 'geo'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'e350'), 'model'] = 'ecoline'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'e250'), 'model'] = 'ecoline'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'e150'), 'model'] = 'ecoline'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'econoline'), 'model'] = 'ecoline'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'f350'), 'model'] = 'f'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'f250'), 'model'] = 'f'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'f150'), 'model'] = 'f'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'shelby500'), 'model'] = 'mustang'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'taurus x'), 'model'] = 'taurus'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'expedit'), 'model'] = 'expedition'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'expeditn'), 'model'] = 'expedition'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'excurs'), 'model'] = 'excursion'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'freestar'), 'model'] = 'freestyle'
data_train.loc[(data_train['make'] == 'ford') & (data_train['model'] == 'police'), 'model'] = 'ranger'
data_train.loc[(data_train['make'] == 'gmc') & (data_train['model'] == 'subrbn'), 'model'] = 'suburban'
data_train.loc[(data_train['make'] == 'gmc') & (data_train['model'] == 'siera'), 'model'] = 'sierra'
data_train.loc[(data_train['make'] == 'gmc') & (data_train['model'] == 'sr'), 'model'] = 'sierra'
data_train.loc[(data_train['make'] == 'honda') & (data_train['model'] == 'cr-z'), 'model'] = 'cr-v'
data_train.loc[(data_train['make'] == 'honda') & (data_train['model'] == 'ridgelin'), 'model'] = 'ridgeline'
data_train.loc[(data_train['make'] == 'hummer') & (data_train['model'] == 'h3t'), 'model'] = 'h3'
data_train.loc[(data_train['make'] == 'hyundai') & (data_train['model'] == 'xg300'), 'model'] = 'xg350'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'ex35'), 'model'] = 'ex'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'fx35'), 'model'] = 'fx'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'fx45'), 'model'] = 'fx'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'fx50'), 'model'] = 'fx'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'g20'), 'model'] = 'g'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'g37'), 'model'] = 'g'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'g35'), 'model'] = 'g'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'i30'), 'model'] = 'i'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'i35'), 'model'] = 'i'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'j30'), 'model'] = 'i'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'jx'), 'model'] = 'i'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'm35'), 'model'] = 'm'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'm37'), 'model'] = 'm'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'm45'), 'model'] = 'm'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'm56'), 'model'] = 'm'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'q45'), 'model'] = 'q'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'q50'), 'model'] = 'q'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'q60'), 'model'] = 'q'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'q70'), 'model'] = 'q'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'qx4'), 'model'] = 'qx'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'qx50'), 'model'] = 'qx'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'qx56'), 'model'] = 'qx'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'qx60'), 'model'] = 'qx'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'qx70'), 'model'] = 'qx'
data_train.loc[(data_train['make'] == 'infiniti') & (data_train['model'] == 'qx80'), 'model'] = 'qx'
data_train.loc[(data_train['make'] == 'jeep') & (data_train['model'] == 'gr'), 'model'] = 'cherokee'
data_train.loc[(data_train['make'] == 'jeep') & (data_train['model'] == 'grand cherokee'), 'model'] = 'cherokee'
data_train.loc[(data_train['make'] == 'jeep') & (data_train['model'] == 'grand'), 'model'] = 'cherokee'
data_train.loc[(data_train['make'] == 'landrover') & (data_train['model'] == 'rr'), 'model'] = 'rangerover'
data_train.loc[(data_train['make'] == 'landrover') & (data_train['model'] == 'rrs'), 'model'] = 'rangerover'
data_train.loc[(data_train['make'] == 'landrover') & (data_train['model'] == 'range'), 'model'] = 'rangerover'
data_train.loc[(data_train['make'] == 'lexus') & (data_train['model'] == 'esh'), 'model'] = 'es'
data_train.loc[(data_train['make'] == 'lexus') & (data_train['model'] == 'hsh'), 'model'] = 'es'
data_train.loc[(data_train['make'] == 'lexus') & (data_train['model'] == 'is c'), 'model'] = 'is'
data_train.loc[(data_train['make'] == 'lexus') & (data_train['model'] == 'is f'), 'model'] = 'is'
data_train.loc[(data_train['make'] == 'lexus') & (data_train['model'] == 'rc f'), 'model'] = 'rc'
data_train.loc[(data_train['make'] == 'lexus') & (data_train['model'] == 'rxh'), 'model'] = 'rx'
data_train.loc[(data_train['make'] == 'linkoln') & (data_train['model'] == 'mark lt'), 'model'] = 'mark'
data_train.loc[(data_train['make'] == 'linkoln') & (data_train['model'] == 'navigator l'), 'model'] = 'navigator'
data_train.loc[(data_train['make'] == 'mazda') & (data_train['model'] == 'mazda3'), 'model'] = '3'
data_train.loc[(data_train['make'] == 'mazda') & (data_train['model'] == 'mazda2'), 'model'] = '2'
data_train.loc[(data_train['make'] == 'mazda') & (data_train['model'] == 'mazda5'), 'model'] = '5'
data_train.loc[(data_train['make'] == 'mazda') & (data_train['model'] == 'mazda6'), 'model'] = '6'
data_train.loc[(data_train['make'] == 'mazda') & (data_train['model'] == '626'), 'model'] = '6'
data_train.loc[(data_train['make'] == 'mazda') & (data_train['model'] == 'b2300'), 'model'] = 'b'
data_train.loc[(data_train['make'] == 'mazda') & (data_train['model'] == 'mazdaspeed3'), 'model'] = '3'
data_train.loc[(data_train['make'] == 'mazda') & (data_train['model'] == 'protege5'), 'model'] = 'protege'
data_train.loc[(data_train['make'] == 'mazda') & (data_train['model'] == 'rx-8'), 'model'] = 'rx8'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == '300e'), 'model'] = '300'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == '420sel'), 'model'] = '420'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'bectric drive'), 'model'] = 'b'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'b200'), 'model'] = 'b'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'c230'), 'model'] = 'c'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'c230wz'), 'model'] = 'c'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'c240s'), 'model'] = 'c'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'c240w'), 'model'] = 'c'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'cl55'), 'model'] = 'cl'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'e300dt'), 'model'] = 'e'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'g500'), 'model'] = 'g'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'g55'), 'model'] = 'g'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'ml55 amg'), 'model'] = 'ml'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'm'), 'model'] = 'ml'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 's55'), 'model'] = 's'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'sl55'), 'model'] = 'sl'
data_train.loc[(data_train['make'] == 'mercedes') & (data_train['model'] == 'sls amg'), 'model'] = 'sls'
data_train.loc[(data_train['make'] == 'mercury') & (data_train['model'] == 'montego'), 'model'] = 'monterey'
data_train.loc[(data_train['make'] == 'mercury') & (data_train['model'] == 'mountnr'), 'model'] = 'monterey'
data_train.loc[(data_train['make'] == 'mercury') & (data_train['model'] == 'mountaineer'), 'model'] = 'monterey'
data_train.loc[(data_train['make'] == 'mitsubishi') & (data_train['model'] == 'lancerolution'), 'model'] = 'lancer'
data_train.loc[(data_train['make'] == 'mitsubishi') & (data_train['model'] == 'lancerback'), 'model'] = 'lancer'
data_train.loc[(data_train['make'] == 'nissan') & (data_train['model'] == '200sx'), 'model'] = 'z'
data_train.loc[(data_train['make'] == 'nissan') & (data_train['model'] == '300zx'), 'model'] = 'z'
data_train.loc[(data_train['make'] == 'nissan') & (data_train['model'] == '350z'), 'model'] = 'z'
data_train.loc[(data_train['make'] == 'nissan') & (data_train['model'] == '370z'), 'model'] = 'z'
data_train.loc[(data_train['make'] == 'nissan') & (data_train['model'] == 'nv200'), 'model'] = 'nv'
data_train.loc[(data_train['make'] == 'nissan') & (data_train['model'] == 'pathfind'), 'model'] = 'pathfinder'
data_train.loc[(data_train['make'] == 'porsche') & (data_train['model'] == 'cayman s'), 'model'] = 'cayman'
data_train.loc[(data_train['make'] == 'saturn') & (data_train['model'] == 'l300'), 'model'] = 'l'
data_train.loc[(data_train['make'] == 'toyota') & (data_train['model'] == 'prius c'), 'model'] = 'prius'
data_train.loc[(data_train['make'] == 'toyota') & (data_train['model'] == 'prius v'), 'model'] = 'prius'
data_train.loc[(data_train['make'] == 'volkswagen') & (data_train['model'] == 'gli'), 'model'] = 'gti'
data_train.loc[(data_train['make'] == 'volkswagen') & (data_train['model'] == 'golf r'), 'model'] = 'golf'
data_train.loc[(data_train['make'] == 'volkswagen') & (data_train['model'] == 'golfi'), 'model'] = 'golf'
data_train.loc[(data_train['make'] == 'volkswagen') & (data_train['model'] == 'jettawagen'), 'model'] = 'jetta'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 'c30'), 'model'] = 'c'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 'c70'), 'model'] = 'c'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 's40'), 'model'] = 's'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 's60'), 'model'] = 's'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 's70'), 'model'] = 's'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 's80'), 'model'] = 's'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 's90'), 'model'] = 's'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 'v40'), 'model'] = 'v'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 'v50'), 'model'] = 'v'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 'v60'), 'model'] = 'v'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 'v70'), 'model'] = 'v'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 'xc60'), 'model'] = 'xc'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 'xc70'), 'model'] = 'xc'
data_train.loc[(data_train['make'] == 'volvo') & (data_train['model'] == 'xc90'), 'model'] = 'xc'
data_train['model'] = data_train['model'].str.replace(' ', '')


for x in data_train['make'].sort_values().unique():
    print(x)
    print(data_train.loc[data_train['make'] == x, 'model'].sort_values().unique())
    print()
#display(data_train['model'].sort_values().unique())

acura
['cl' 'el' 'ilx' 'integra' 'legend' 'mdx' 'rdx' 'rl' 'rlx' 'rsx' 'tl'
 'tsx' 'zdx']

airstream
['interstate']

astonmartin
['db9' 'rapide' 'vantage']

audi
['a3' 'a4' 'a5' 'a6' 'a7' 'a8' 'allroad' 'q3' 'q5' 'q7' 's4' 's5' 's6'
 's7' 's8' 'tt' nan]

bentley
['continental' 'flyingspur']

bmw
['1' '2' '3' '4' '5' '6' '7' '8' 'alp' 'i8' 'm' 'm3' 'm4' 'm5' 'm6' 'x1'
 'x3' 'x4' 'x5' 'x6' 'z3' 'z4' nan]

buick
['century' 'enclave' 'encore' 'lacrosse' 'lesabre' 'lucerne' 'parkavenue'
 'rainier' 'regal' 'rendezvous' 'riviera' 'roadmaster' 'terraza' 'verano']

cadillac
['ats' 'catera' 'cts' 'deville' 'dts' 'eldorado' 'elr' 'escalade'
 'fleetwood' 'seville' 'srx' 'sts' 'xlr' 'xts']

chevrolet
['astro' 'avalanche' 'aveo' 'blazer' 'camaro' 'caprice' 'captiva'
 'cavalier' 'ck' 'classic' 'cobalt' 'colorado' 'comm' 'corsica' 'corvette'
 'cruze' 'equinox' 'express' 'hhr' 'impala' 'ker' 'lumina' 'malibu'
 'montelo' 'optra' 'prizm' 'silverado' 'sonic' 'spark' 'ssr' 'suburban'
 'tahoe' 'trailblazer'

Cleaning "Model" column

In [9]:
data_train.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate,sell_year,sell_month,sell_day,age
0,2011,ford,edge,sel,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,2015-06-02 02:30:00+00:00,2015,6,2,4
1,2014,ford,fusion,se,sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,2015-02-25 02:00:00+00:00,2015,2,25,1
2,2012,nissan,sentra,2.0 sl,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,2015-06-10 02:30:00+00:00,2015,6,10,3
3,2003,hummer,h2,base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,2015-06-17 03:00:00+00:00,2015,6,17,12
4,2007,ford,fusion,sel,sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,2015-02-03 04:00:00+00:00,2015,2,3,8


Filling out null-values

In [12]:
data_train['model'].fillna('other', inplace=True)
data_train['trim'].fillna(data_train['trim'].mode().iloc[0], inplace=True)
data_train['transmission'].fillna(data_train['transmission'].mode().iloc[0], inplace=True)
data_train['color'].fillna(data_train['color'].mode().iloc[0], inplace=True)
data_train['interior'].fillna(data_train['interior'].mode().iloc[0], inplace=True)
data_train['body'].fillna(data_train['body'].mode().iloc[0], inplace=True)
data_train['condition'] = data_train['condition'].fillna(data_train.groupby(['make', 'model'])['condition'].transform('mean'))
data_train['odometer'] = data_train['odometer'].fillna(data_train.groupby(['make', 'model'])['odometer'].transform('mean'))
data_train['condition'] = data_train['condition'].fillna(data_train.groupby('make')['condition'].transform('mean'))
data_train.replace([np.inf, -np.inf], np.nan, inplace=True)

In [13]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype                
---  ------        --------------   -----                
 0   year          440236 non-null  int64                
 1   make          440236 non-null  object               
 2   model         440236 non-null  object               
 3   trim          440236 non-null  object               
 4   body          440236 non-null  object               
 5   transmission  440236 non-null  object               
 6   vin           440236 non-null  object               
 7   state         440236 non-null  object               
 8   condition     440236 non-null  float64              
 9   odometer      440236 non-null  float64              
 10  color         440236 non-null  object               
 11  interior      440236 non-null  object               
 12  seller        440236 non-null  object               
 13  sellingprice  

## Model trials

### Data Preparation

In [14]:
data_train.drop(['vin', 'saledate'], axis=1, inplace=True)

In [15]:
features_train = data_train.drop(['sellingprice', 'sell_year'], axis=1)
target_train = data_train['sellingprice']

In [16]:
categorical_features = features_train.select_dtypes(exclude='number').columns.tolist()
print(f'There are {len(categorical_features)} categorical features:', '\n')
print(categorical_features)
print()
numerical_features = features_train.select_dtypes(include='number').columns.tolist()
print(f'There are {len(numerical_features)} numerical features:', '\n')
print(numerical_features)
print()

There are 9 categorical features: 

['make', 'model', 'trim', 'body', 'transmission', 'state', 'color', 'interior', 'seller']

There are 6 numerical features: 

['year', 'condition', 'odometer', 'sell_month', 'sell_day', 'age']



In [17]:
categorical_features = ['year', 'make', 'model', 'trim', 'body', 'transmission', 'state', 'color', 'interior', 'seller',  'age']

In [18]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(features_train[['year', 'make', 'model', 'trim', 'body', 'transmission', 'state', 'color', 'interior', 'seller', 'age']])
features_train[['year', 'make', 'model', 'trim', 'body', 'transmission', 'state', 'color', 'interior', 'seller', 'age']] = encoder.transform(features_train[['year', 'make', 'model', 'trim', 'body', 'transmission', 'state', 'color', 'interior', 'seller', 'age']])

scaler = StandardScaler()
scaler.fit(features_train[['condition', 'odometer']])
features_train[['condition', 'odometer']] = scaler.transform(features_train[['condition', 'odometer']])

In [19]:
# features_train = features_train.reset_index()
# target_train = target_train.reset_index()

In [20]:
features_train.isna().sum()

year            0
make            0
model           0
trim            0
body            0
transmission    0
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
sell_month      0
sell_day        0
age             0
dtype: int64

### Decision Tree

In [23]:
model_tree  = DecisionTreeRegressor(random_state=777)

tree_pipeline = Pipeline(steps=[('tree', model_tree)])

parameters = {
    'tree__min_samples_leaf': range(3, 5, 1),
    'tree__max_depth': range(1, 16, 2),
}

clf = GridSearchCV(tree_pipeline,
                   param_grid=parameters,
                   scoring='neg_mean_absolute_error',
                   return_train_score=True,
                   cv=2)
clf.fit(features_train, target_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('tree',
                                        DecisionTreeRegressor(random_state=777))]),
             param_grid={'tree__max_depth': range(1, 16, 2),
                         'tree__min_samples_leaf': range(3, 5)},
             return_train_score=True, scoring='neg_mean_absolute_error')

### CatBoost

In [24]:
CBC = CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE')

parameters = {'depth'         : [1,2,3,4,5],
                 'learning_rate' : [0.01, 0.02, 0.03, 0.04, 0.05],
                  'iterations'    : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
                 }

Grid_CBC = GridSearchCV(estimator=CBC, param_grid = parameters, cv = 2, n_jobs=None)
Grid_CBC.fit(features_train, target_train)

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid_CBC.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBC.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBC.best_params_)

0:	learn: 9740.5275028	total: 172ms	remaining: 1.55s
1:	learn: 9711.6654426	total: 181ms	remaining: 725ms
2:	learn: 9683.1718122	total: 190ms	remaining: 443ms
3:	learn: 9655.4780313	total: 199ms	remaining: 298ms
4:	learn: 9627.8064444	total: 207ms	remaining: 207ms
5:	learn: 9600.5887212	total: 216ms	remaining: 144ms
6:	learn: 9573.8866622	total: 225ms	remaining: 96.4ms
7:	learn: 9547.7784554	total: 233ms	remaining: 58.3ms
8:	learn: 9521.6495227	total: 242ms	remaining: 26.9ms
9:	learn: 9495.9789569	total: 251ms	remaining: 0us
0:	learn: 9703.6884247	total: 9.65ms	remaining: 86.9ms
1:	learn: 9674.8637327	total: 18.6ms	remaining: 74.3ms
2:	learn: 9646.4167431	total: 26.9ms	remaining: 62.9ms
3:	learn: 9618.7719630	total: 35.4ms	remaining: 53.1ms
4:	learn: 9591.1367440	total: 43.4ms	remaining: 43.4ms
5:	learn: 9563.9779274	total: 51.7ms	remaining: 34.4ms
6:	learn: 9537.2798199	total: 59.9ms	remaining: 25.7ms
7:	learn: 9511.2158600	total: 68.5ms	remaining: 17.1ms
8:	learn: 9485.4221353	total:

### optuna trials

In [25]:
def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(features_train, target_train, test_size=0.3)
    param = {
        "loss_function": trial.suggest_categorical("loss_function", ["RMSE", "MAE"]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 20),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20),  
    }
    # Conditional Hyper-Parameters
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    reg = CatBoostRegressor(**param)
    reg.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=100)
    y_pred = reg.predict(X_test)
    score = mean_squared_error(y_test, y_pred, squared=False)
    return score

In [26]:
study = optuna.create_study(sampler=TPESampler(), direction="minimize")
study.optimize(objective, n_trials=10, timeout=600) # Run for 10 minutes
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-04-07 17:22:46,309] A new study created in memory with name: no-name-46faf606-eacf-4ffb-912d-9c74b8375a2d
[I 2023-04-07 17:22:56,414] Trial 0 finished with value: 9510.73444384037 and parameters: {'loss_function': 'RMSE', 'learning_rate': 0.0009160264651747462, 'l2_leaf_reg': 0.02329061462722597, 'colsample_bylevel': 0.01638114197605771, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 12, 'one_hot_max_size': 15}. Best is trial 0 with value: 9510.73444384037.
[I 2023-04-07 17:23:27,528] Trial 1 finished with value: 6642.251209339705 and parameters: {'loss_function': 'RMSE', 'learning_rate': 0.007390357190081049, 'l2_leaf_reg': 0.029424012847669868, 'colsample_bylevel': 0.0710949593248897, 'depth': 4, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'min_data_in_leaf': 15, 'one_hot_max_size': 5, 'bagging_temperature': 2.2079804060055386}. Best is trial 1 with value: 6642.251209339705.
[I 2023-04-07 17:23:42,696] Trial 2 finished with va

Number of completed trials: 10
Best trial:
	Best Score: 5327.789212273994
	Best Params: 
    loss_function: MAE
    learning_rate: 0.06720703897167796
    l2_leaf_reg: 0.21764742567391365
    colsample_bylevel: 0.0726829183728953
    depth: 10
    boosting_type: Plain
    bootstrap_type: MVS
    min_data_in_leaf: 15
    one_hot_max_size: 13


## Best model selection

In [27]:
print('Decision tree results')
display(clf.best_score_)
print(clf.best_params_)
print()
print('Catboost results')
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid_CBC.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBC.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBC.best_params_)
print()
print('Optuna results')
print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Decision tree results


-2027.8201916399728

{'tree__max_depth': 15, 'tree__min_samples_leaf': 4}

Catboost results
 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.6938030987384223

 The best parameters across ALL searched params:
 {'depth': 5, 'iterations': 100, 'learning_rate': 0.05}

Optuna results
	Best Score: 5327.789212273994
	Best Params: 
    loss_function: MAE
    learning_rate: 0.06720703897167796
    l2_leaf_reg: 0.21764742567391365
    colsample_bylevel: 0.0726829183728953
    depth: 10
    boosting_type: Plain
    bootstrap_type: MVS
    min_data_in_leaf: 15
    one_hot_max_size: 13


## test sample

In [29]:
data_test = pd.read_csv(r'C:\Users\90sds\Desktop\Python\Yandex\Masterskaya\data\test.csv')

In [30]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110060 entries, 0 to 110059
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          110060 non-null  int64  
 1   make          107999 non-null  object 
 2   model         107981 non-null  object 
 3   trim          107946 non-null  object 
 4   body          107466 non-null  object 
 5   transmission  97048 non-null   object 
 6   vin           110060 non-null  object 
 7   state         110060 non-null  object 
 8   condition     107681 non-null  float64
 9   odometer      110041 non-null  float64
 10  color         109902 non-null  object 
 11  interior      109902 non-null  object 
 12  seller        110060 non-null  object 
 13  saledate      110060 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 11.8+ MB


In [31]:
data_test.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,saledate
0,2005,Cadillac,CTS,Base,Sedan,automatic,1g6dp567450124779,ca,2.7,116970.0,silver,black,lexus of stevens creek,Wed Jan 14 2015 04:30:00 GMT-0800 (PST)
1,2014,GMC,Savana Cargo,2500,Van,NaN,1gtw7fca7e1902207,pa,4.4,6286.0,white,gray,u-haul,Fri Feb 27 2015 01:00:00 GMT-0800 (PST)
2,2013,Nissan,Murano,S,SUV,automatic,jn8az1mw6dw303497,oh,4.6,11831.0,gray,black,nissan-infiniti lt,Tue Feb 24 2015 01:30:00 GMT-0800 (PST)
3,2013,Chevrolet,Impala,LS Fleet,Sedan,automatic,2g1wf5e34d1160703,fl,2.3,57105.0,silver,black,onemain rem/auto club of miami inc dba north dad,Fri Mar 06 2015 02:00:00 GMT-0800 (PST)
4,2013,Nissan,Titan,SV,Crew Cab,automatic,1n6aa0ec3dn301209,tn,2.9,31083.0,black,black,nissan north america inc.,Wed Jun 03 2015 03:30:00 GMT-0700 (PDT)


In [32]:
data_test['saledate'] = pd.to_datetime(data_test['saledate'], format='%a %b %d %Y %H:%M:%S %Z', exact=False)
data_test['sell_year'] = data_test['saledate'].dt.year
data_test['sell_month'] = data_test['saledate'].dt.month
data_test['sell_day'] = data_test['saledate'].dt.day
data_test['age'] = data_test['sell_year'] - data_train['year']

In [33]:
for x in data_test.select_dtypes(include="object"):
    data_test[x] = data_test[x].str.lower()
    
make_dupl = [' tk', ' truck', ' ', '-b', '-benz']
for y in make_dupl:
    data_test['make'] = data_test['make'].str.replace(y, '')

data_test['make'] = data_test['make'].str.replace('mercedesenz', 'mercedes')    
data_test['make'] = data_test['make'].str.replace('vw', 'volkswagen')
data_test['make'].fillna('other', inplace=True)

In [34]:
display(data_test['make'].sort_values().unique())
print()
display(data_test['make'].value_counts())

array(['acura', 'astonmartin', 'audi', 'bentley', 'bmw', 'buick',
       'cadillac', 'chev', 'chevrolet', 'chrysler', 'daewoo', 'dodge',
       'ferrari', 'fiat', 'fisker', 'ford', 'geo', 'gmc', 'honda',
       'hummer', 'hyundai', 'infiniti', 'isuzu', 'jaguar', 'jeep', 'kia',
       'landrover', 'lexus', 'lincoln', 'lotus', 'maserati', 'mazda',
       'mercedes', 'mercury', 'mini', 'mitsubishi', 'nissan',
       'oldsmobile', 'other', 'plymouth', 'pontiac', 'porsche', 'ram',
       'rolls-royce', 'saab', 'saturn', 'scion', 'smart', 'subaru',
       'suzuki', 'tesla', 'toyota', 'volkswagen', 'volvo'], dtype=object)

ford           18467
chevrolet      11801
nissan         10485
toyota          7728
dodge           6188
honda           5382
hyundai         4410
bmw             4136
kia             3571
mercedes        3471
chrysler        3459
jeep            3069
infiniti        3002
volkswagen      2515
lexus           2430
other           2061
gmc             2059
mazda           1676
cadillac        1496
lincoln         1153
acura           1151
audi            1124
buick           1019
subaru          1015
ram              889
pontiac          866
mitsubishi       806
volvo            765
mini             674
saturn           544
mercury          434
landrover        371
scion            318
jaguar           297
porsche          280
suzuki           222
fiat             181
hummer           174
saab              93
oldsmobile        88
smart             81
isuzu             39
bentley           23
maserati          21
tesla              6
plymouth           4
geo                3
rolls-royce  

In [35]:
model_dupl = [' sports wagon', 'v8 ', ' quattro', ' gtc', ' gt', ' speed', ' supersports', ' series', ' gran turismo',
              ' gran coupe', 'activehybrid ', ' coupe', ' hybrid', ' esv', ' ext', ' wagon', ' limited', '3500hd', ' sport', 
             ' classic', ' 2500hd', ' 1500hd', 'black diamond ', ' 1500', ' 2500', ' stingray', 's-10 ', ' maxx', '/z71',
             ' cargo', ' 3500', ' ev', ' cruiser', 'pickup', ' van', '-150', '-250', '-350', '-series', ' victoria', 'energi',
             ' heritage', ' super duty', ' st', ' hundred', ' connect', ' svt cobra', ' svt lightning', '-450', ' el', 'trac',
             ' xl', ' xuv', ' denali', ' crosstour', ' del sol', ' touring', ' sut', ' fe', ' convertible', ' sedan', ' ii',
             ' roveroque', ' 250', ' 350', ' 330', ' 400', ' 430', ' 450h', ' 460', ' 470', ' 250h', ' 300', ' 200h', ' 300h',
             ' 600h l', ' 570', ' 450', ' viii', ' car', ' truck', ' miata', 'mazdaspeed ', ' mazdaspeed', '-class', ' marquis',
             'cooper ', ' spyder', ' passenger', ' select', ' note', ' crosstt', ' ciera', ' supreme', ' royale', ' am', ' prix',
             ' sv6', '/', ' tradesman', 'b9 ', ' wrx', ' crosstrek', ' vitara', ' solara', ' plug-in', ' 2', 'new ', ' gli']
for y in model_dupl:
    data_test['model'] = data_test['model'].str.replace(y, '')

data_test['model'] = data_test['model'].str.replace('rs 7', 's7')
data_test['model'] = data_test['model'].str.replace('r8', 's8')
data_test['model'] = data_test['model'].str.replace('rs 6', 's6')
data_test['model'] = data_test['model'].str.replace('cabriolet', 'tt')
data_test['model'] = data_test['model'].str.replace('tts', 'tt')
data_test['model'] = data_test['model'].str.replace('rs 4', 's4')
data_test['model'] = data_test['model'].str.replace('sq5', 'q5')
data_test['model'] = data_test['model'].str.replace('rs 5', 's5')
data_test['model'] = data_test['model'].str.replace('tt rs', 'tt')
data_test['model'] = data_test['model'].str.replace('continental flying spur', 'flying spur')
data_test['model'] = data_test['model'].str.replace('x6 m', 'x6')
data_test['model'] = data_test['model'].str.replace('750i', '7')
data_test['model'] = data_test['model'].str.replace('750li', '7')
data_test['model'] = data_test['model'].str.replace('320i', '3')
data_test['model'] = data_test['model'].str.replace('x5 m', 'x5')
data_test['model'] = data_test['model'].str.replace('750lxi', '7')
data_test['model'] = data_test['model'].str.replace('z4 m', 'z4')
data_test['model'] = data_test['model'].str.replace('328i', '3')
data_test['model'] = data_test['model'].str.replace('323i', '3')
data_test['model'] = data_test['model'].str.replace('x6 m', 'x6')
data_test['model'] = data_test['model'].str.replace('cts-v', 'cts')
data_test['model'] = data_test['model'].str.replace('sts-v', 'sts')
data_test['model'] = data_test['model'].str.replace('s-10', 'blazer')
data_test['model'] = data_test['model'].str.replace('s10', 'blazer')
data_test['model'] = data_test['model'].str.replace('c/k', 'silverado')
data_test['model'] = data_test['model'].str.replace('uplandr', 'uplander')
data_test.loc[(data_test['make'] == 'chevrolet') & (data_test['model'] == '2500'), 'model'] = 'silverado'
data_test.loc[(data_test['make'] == 'chevrolet') & (data_test['model'] == 'g2500'), 'model'] = 'silverado'
data_test.loc[(data_test['make'] == 'chevrolet') & (data_test['model'] == 'g3500'), 'model'] = 'silverado'
data_test.loc[(data_test['make'] == 'chevrolet') & (data_test['model'] == '3500'), 'model'] = 'silverado'
data_test.loc[(data_test['make'] == 'chevrolet') & (data_test['model'] == 'g1500'), 'model'] = 'silverado'
data_test.loc[(data_test['make'] == 'chevrolet') & (data_test['model'] == '1500'), 'model'] = 'silverado'
data_test.loc[(data_test['make'] == 'chevrolet') & (data_test['model'] == 'capt'), 'model'] = 'captiva'
data_test.loc[(data_test['make'] == 'chevrolet') & (data_test['model'] == 'ss'), 'model'] = 'ssr'
data_test.loc[(data_test['make'] == 'chrysler') & (data_test['model'] == '300m'), 'model'] = '300'
data_test.loc[(data_test['make'] == 'chrysler') & (data_test['model'] == 'twn&country'), 'model'] = 'twn/cntry'
data_test.loc[(data_test['make'] == 'chrysler') & (data_test['model'] == 'town'), 'model'] = 'twn/cntry'
data_test.loc[(data_test['make'] == 'chrysler') & (data_test['model'] == 'town and country'), 'model'] = 'twn/cntry'
data_test.loc[(data_test['make'] == 'dodge') & (data_test['model'] == 'grand'), 'model'] = 'gr'
data_test.loc[(data_test['make'] == 'dodge') & (data_test['model'] == 'grand caravan'), 'model'] = 'gr'
data_test.loc[(data_test['make'] == 'dodge') & (data_test['model'] == 'ram3500'), 'model'] = 'ram'
data_test.loc[data_test['make'] == 'ferrari', 'model'] = 'ferrari'
data_test.loc[data_test['make'] == 'fiat', 'model'] = '500'
data_test.loc[data_test['make'] == 'geo', 'model'] = 'geo'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'e350'), 'model'] = 'ecoline'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'e250'), 'model'] = 'ecoline'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'e150'), 'model'] = 'ecoline'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'econoline'), 'model'] = 'ecoline'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'f350'), 'model'] = 'f'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'f250'), 'model'] = 'f'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'f150'), 'model'] = 'f'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'shelby500'), 'model'] = 'mustang'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'taurus x'), 'model'] = 'taurus'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'expedit'), 'model'] = 'expedition'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'expeditn'), 'model'] = 'expedition'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'excurs'), 'model'] = 'excursion'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'freestar'), 'model'] = 'freestyle'
data_test.loc[(data_test['make'] == 'ford') & (data_test['model'] == 'police'), 'model'] = 'ranger'
data_test.loc[(data_test['make'] == 'gmc') & (data_test['model'] == 'subrbn'), 'model'] = 'suburban'
data_test.loc[(data_test['make'] == 'gmc') & (data_test['model'] == 'siera'), 'model'] = 'sierra'
data_test.loc[(data_test['make'] == 'gmc') & (data_test['model'] == 'sr'), 'model'] = 'sierra'
data_test.loc[(data_test['make'] == 'honda') & (data_test['model'] == 'cr-z'), 'model'] = 'cr-v'
data_test.loc[(data_test['make'] == 'honda') & (data_test['model'] == 'ridgelin'), 'model'] = 'ridgeline'
data_test.loc[(data_test['make'] == 'hummer') & (data_test['model'] == 'h3t'), 'model'] = 'h3'
data_test.loc[(data_test['make'] == 'hyundai') & (data_test['model'] == 'xg300'), 'model'] = 'xg350'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'ex35'), 'model'] = 'ex'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'fx35'), 'model'] = 'fx'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'fx45'), 'model'] = 'fx'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'fx50'), 'model'] = 'fx'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'g20'), 'model'] = 'g'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'g37'), 'model'] = 'g'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'g35'), 'model'] = 'g'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'i30'), 'model'] = 'i'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'i35'), 'model'] = 'i'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'j30'), 'model'] = 'i'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'jx'), 'model'] = 'i'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'm35'), 'model'] = 'm'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'm37'), 'model'] = 'm'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'm45'), 'model'] = 'm'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'm56'), 'model'] = 'm'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'q45'), 'model'] = 'q'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'q50'), 'model'] = 'q'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'q60'), 'model'] = 'q'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'q70'), 'model'] = 'q'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'qx4'), 'model'] = 'qx'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'qx50'), 'model'] = 'qx'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'qx56'), 'model'] = 'qx'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'qx60'), 'model'] = 'qx'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'qx70'), 'model'] = 'qx'
data_test.loc[(data_test['make'] == 'infiniti') & (data_test['model'] == 'qx80'), 'model'] = 'qx'
data_test.loc[(data_test['make'] == 'jeep') & (data_test['model'] == 'gr'), 'model'] = 'cherokee'
data_test.loc[(data_test['make'] == 'jeep') & (data_test['model'] == 'grand cherokee'), 'model'] = 'cherokee'
data_test.loc[(data_test['make'] == 'jeep') & (data_test['model'] == 'grand'), 'model'] = 'cherokee'
data_test.loc[(data_test['make'] == 'landrover') & (data_test['model'] == 'rr'), 'model'] = 'rangerover'
data_test.loc[(data_test['make'] == 'landrover') & (data_test['model'] == 'rrs'), 'model'] = 'rangerover'
data_test.loc[(data_test['make'] == 'landrover') & (data_test['model'] == 'range'), 'model'] = 'rangerover'
data_test.loc[(data_test['make'] == 'lexus') & (data_test['model'] == 'esh'), 'model'] = 'es'
data_test.loc[(data_test['make'] == 'lexus') & (data_test['model'] == 'hsh'), 'model'] = 'es'
data_test.loc[(data_test['make'] == 'lexus') & (data_test['model'] == 'is c'), 'model'] = 'is'
data_test.loc[(data_test['make'] == 'lexus') & (data_test['model'] == 'is f'), 'model'] = 'is'
data_test.loc[(data_test['make'] == 'lexus') & (data_test['model'] == 'rc f'), 'model'] = 'rc'
data_test.loc[(data_test['make'] == 'lexus') & (data_test['model'] == 'rxh'), 'model'] = 'rx'
data_test.loc[(data_test['make'] == 'linkoln') & (data_test['model'] == 'mark lt'), 'model'] = 'mark'
data_test.loc[(data_test['make'] == 'linkoln') & (data_test['model'] == 'navigator l'), 'model'] = 'navigator'
data_test.loc[(data_test['make'] == 'mazda') & (data_test['model'] == 'mazda3'), 'model'] = '3'
data_test.loc[(data_test['make'] == 'mazda') & (data_test['model'] == 'mazda2'), 'model'] = '2'
data_test.loc[(data_test['make'] == 'mazda') & (data_test['model'] == 'mazda5'), 'model'] = '5'
data_test.loc[(data_test['make'] == 'mazda') & (data_test['model'] == 'mazda6'), 'model'] = '6'
data_test.loc[(data_test['make'] == 'mazda') & (data_test['model'] == '626'), 'model'] = '6'
data_test.loc[(data_test['make'] == 'mazda') & (data_test['model'] == 'b2300'), 'model'] = 'b'
data_test.loc[(data_test['make'] == 'mazda') & (data_test['model'] == 'mazdaspeed3'), 'model'] = '3'
data_test.loc[(data_test['make'] == 'mazda') & (data_test['model'] == 'protege5'), 'model'] = 'protege'
data_test.loc[(data_test['make'] == 'mazda') & (data_test['model'] == 'rx-8'), 'model'] = 'rx8'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == '300e'), 'model'] = '300'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == '420sel'), 'model'] = '420'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'bectric drive'), 'model'] = 'b'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'b200'), 'model'] = 'b'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'c230'), 'model'] = 'c'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'c230wz'), 'model'] = 'c'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'c240s'), 'model'] = 'c'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'c240w'), 'model'] = 'c'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'cl55'), 'model'] = 'cl'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'e300dt'), 'model'] = 'e'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'g500'), 'model'] = 'g'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'g55'), 'model'] = 'g'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'ml55 amg'), 'model'] = 'ml'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'm'), 'model'] = 'ml'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 's55'), 'model'] = 's'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'sl55'), 'model'] = 'sl'
data_test.loc[(data_test['make'] == 'mercedes') & (data_test['model'] == 'sls amg'), 'model'] = 'sls'
data_test.loc[(data_test['make'] == 'mercury') & (data_test['model'] == 'montego'), 'model'] = 'monterey'
data_test.loc[(data_test['make'] == 'mercury') & (data_test['model'] == 'mountnr'), 'model'] = 'monterey'
data_test.loc[(data_test['make'] == 'mercury') & (data_test['model'] == 'mountaineer'), 'model'] = 'monterey'
data_test.loc[(data_test['make'] == 'mitsubishi') & (data_test['model'] == 'lancerolution'), 'model'] = 'lancer'
data_test.loc[(data_test['make'] == 'mitsubishi') & (data_test['model'] == 'lancerback'), 'model'] = 'lancer'
data_test.loc[(data_test['make'] == 'nissan') & (data_test['model'] == '200sx'), 'model'] = 'z'
data_test.loc[(data_test['make'] == 'nissan') & (data_test['model'] == '300zx'), 'model'] = 'z'
data_test.loc[(data_test['make'] == 'nissan') & (data_test['model'] == '350z'), 'model'] = 'z'
data_test.loc[(data_test['make'] == 'nissan') & (data_test['model'] == '370z'), 'model'] = 'z'
data_test.loc[(data_test['make'] == 'nissan') & (data_test['model'] == 'nv200'), 'model'] = 'nv'
data_test.loc[(data_test['make'] == 'nissan') & (data_test['model'] == 'pathfind'), 'model'] = 'pathfinder'
data_test.loc[(data_test['make'] == 'porsche') & (data_test['model'] == 'cayman s'), 'model'] = 'cayman'
data_test.loc[(data_test['make'] == 'saturn') & (data_test['model'] == 'l300'), 'model'] = 'l'
data_test.loc[(data_test['make'] == 'toyota') & (data_test['model'] == 'prius c'), 'model'] = 'prius'
data_test.loc[(data_test['make'] == 'toyota') & (data_test['model'] == 'prius v'), 'model'] = 'prius'
data_test.loc[(data_test['make'] == 'volkswagen') & (data_test['model'] == 'gli'), 'model'] = 'gti'
data_test.loc[(data_test['make'] == 'volkswagen') & (data_test['model'] == 'golf r'), 'model'] = 'golf'
data_test.loc[(data_test['make'] == 'volkswagen') & (data_test['model'] == 'golfi'), 'model'] = 'golf'
data_test.loc[(data_test['make'] == 'volkswagen') & (data_test['model'] == 'jettawagen'), 'model'] = 'jetta'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 'c30'), 'model'] = 'c'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 'c70'), 'model'] = 'c'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 's40'), 'model'] = 's'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 's60'), 'model'] = 's'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 's70'), 'model'] = 's'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 's80'), 'model'] = 's'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 's90'), 'model'] = 's'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 'v40'), 'model'] = 'v'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 'v50'), 'model'] = 'v'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 'v60'), 'model'] = 'v'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 'v70'), 'model'] = 'v'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 'xc60'), 'model'] = 'xc'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 'xc70'), 'model'] = 'xc'
data_test.loc[(data_test['make'] == 'volvo') & (data_test['model'] == 'xc90'), 'model'] = 'xc'
data_test['model'] = data_test['model'].str.replace(' ', '')


for x in data_test['make'].sort_values().unique():
    print(x)
    print(data_test.loc[data_test['make'] == x, 'model'].sort_values().unique())
    print()
#display(data_test['model'].sort_values().unique())


acura
['cl' 'el' 'ilx' 'integra' 'mdx' 'rdx' 'rl' 'rlx' 'rsx' 'tl' 'tlx' 'tsx'
 'zdx']

astonmartin
['vantage']

audi
['a3' 'a4' 'a5' 'a6' 'a7' 'a8' 'allroad' 'q5' 'q7' 's4' 's5' 's6' 's7'
 's8' 'tt' nan]

bentley
['continental' 'flyingspur']

bmw
['1' '2' '3' '4' '5' '6' '7' 'alp' 'i8' 'm' 'm3' 'm4' 'm5' 'm6' 'x1' 'x3'
 'x4' 'x5' 'x6' 'z3' 'z4' nan]

buick
['allure' 'century' 'enclave' 'encore' 'lacrosse' 'lesabre' 'lucerne'
 'parkavenue' 'rainier' 'regal' 'rendezvous' 'riviera' 'terraza' 'verano']

cadillac
['ats' 'catera' 'cts' 'deville' 'dts' 'eldorado' 'escalade' 'seville'
 'srx' 'sts' 'xlr' 'xts']

chev
['blazer']

chevrolet
['astro' 'avalanche' 'aveo' 'blazer' 'camaro' 'caprice' 'captiva'
 'cavalier' 'ck' 'classic' 'cobalt' 'colorado' 'corvette' 'cruze'
 'equinox' 'express' 'hhr' 'impala' 'ker' 'lumina' 'malibu' 'montelo'
 'prizm' 'silverado' 'sonic' 'spark' 'ssr' 'suburban' 'tahoe'
 'trailblazer' 'traverse' 'uplander' 'venture' 'volt']

chrysler
['200' '300' 'aspen' 'cirrus' 'c

In [36]:
data_test['model'].fillna('other', inplace=True)
data_test['trim'].fillna(data_test['trim'].mode().iloc[0], inplace=True)
data_test['transmission'].fillna(data_test['transmission'].mode().iloc[0], inplace=True)
data_test['color'].fillna(data_test['color'].mode().iloc[0], inplace=True)
data_test['interior'].fillna(data_test['interior'].mode().iloc[0], inplace=True)
data_test['body'].fillna(data_test['body'].mode().iloc[0], inplace=True)
data_test['condition'] = data_test['condition'].fillna(data_test.groupby(['make', 'model'])['condition'].transform('mean'))
data_test['odometer'] = data_test['odometer'].fillna(data_test.groupby(['make', 'model'])['odometer'].transform('mean'))
data_test['condition'] = data_test['condition'].fillna(3)


In [37]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110060 entries, 0 to 110059
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype                
---  ------        --------------   -----                
 0   year          110060 non-null  int64                
 1   make          110060 non-null  object               
 2   model         110060 non-null  object               
 3   trim          110060 non-null  object               
 4   body          110060 non-null  object               
 5   transmission  110060 non-null  object               
 6   vin           110060 non-null  object               
 7   state         110060 non-null  object               
 8   condition     110060 non-null  float64              
 9   odometer      110060 non-null  float64              
 10  color         110060 non-null  object               
 11  interior      110060 non-null  object               
 12  seller        110060 non-null  object               
 13  saledate      

## Final Prediction

In [38]:
features_test = data_test.drop(['vin','saledate','sell_year'], axis=1)

In [39]:
model = CatBoostRegressor()
model.set_params(**study.best_params)
model.fit(features_train, target_train)

0:	learn: 6997.5336350	total: 36.5ms	remaining: 36.4s
1:	learn: 6816.2201636	total: 69.6ms	remaining: 34.7s
2:	learn: 6623.3663161	total: 109ms	remaining: 36.1s
3:	learn: 6618.5903566	total: 141ms	remaining: 35.1s
4:	learn: 6610.7327140	total: 175ms	remaining: 34.9s
5:	learn: 6393.7499758	total: 219ms	remaining: 36.2s
6:	learn: 6241.4196955	total: 255ms	remaining: 36.1s
7:	learn: 6240.8713716	total: 288ms	remaining: 35.7s
8:	learn: 6120.9000160	total: 321ms	remaining: 35.4s
9:	learn: 6120.1490423	total: 353ms	remaining: 34.9s
10:	learn: 6011.5655380	total: 386ms	remaining: 34.7s
11:	learn: 6010.6978414	total: 418ms	remaining: 34.4s
12:	learn: 5888.7082862	total: 452ms	remaining: 34.3s
13:	learn: 5776.0370965	total: 490ms	remaining: 34.5s
14:	learn: 5774.7615878	total: 521ms	remaining: 34.2s
15:	learn: 5633.3200269	total: 568ms	remaining: 34.9s
16:	learn: 5632.0081183	total: 598ms	remaining: 34.6s
17:	learn: 5548.2858504	total: 631ms	remaining: 34.4s
18:	learn: 5479.0394938	total: 665ms

In [41]:
features_test[['year', 'make', 'model', 'trim', 'body', 'transmission', 'state', 'color', 'interior', 'seller', 'age']] = encoder.transform(features_test[['year', 'make', 'model', 'trim', 'body', 'transmission', 'state', 'color', 'interior', 'seller', 'age']])

features_test[['condition', 'odometer']] = scaler.transform(features_test[['condition', 'odometer']])

In [42]:
predicted_test = model.predict(features_test)

In [44]:
submission = pd.DataFrame({'vin': data_test.vin, 'sellingprice': predicted_test})

In [45]:
#submission.to_csv('submission.csv', index=False)
submission.head()

,vin,sellingprice
0,1g6dp567450124779,8577.406171
1,1gtw7fca7e1902207,22371.852315
2,jn8az1mw6dw303497,18763.498124
3,2g1wf5e34d1160703,6054.370075
4,1n6aa0ec3dn301209,20345.870011
